# Notebook to generate dataset of MPC I/O examples

In [ ]:
from mpc import *
from numpy.random import uniform as unf

In [ ]:
model = STM_model_dt_inputs(tire=pacejka); 
# ctrl = MPC_Controller(model, hard_cold_start=True) # with cold start
ctrl = MPC_Controller(model) 
YREF = np.array([4.5, -30*π/180,0,0,0,0,0])  # reference

In [ ]:
# fix v and explore beta/r space
v_lims = (2.0, 8.0)  # m/s # ficed limits
# v_lims = (YREF[0]-1, YREF[0]+1)  # m/s # around ref
beta_lims = (-40*π/180, 40*π/180)
r_lims = (-100*π/180, 100*π/180)

ng = 10 # grid points per dimension

#  check warm start is disabled
assert ctrl.ocp.solver_options.qp_solver_warm_start == 0, f'warm start should be disabled, got {ctrl.ocp.solver_options.qp_solver_warm_start}'

# create a 3d grid of initial conditions
v_grid = np.linspace(v_lims[0], v_lims[1], ng)
beta_grid = np.linspace(beta_lims[0], beta_lims[1], ng)
r_grid = np.linspace(r_lims[0], r_lims[1], ng)

deltas = np.zeros((ng, ng, ng))
fxs = np.zeros((ng, ng, ng))
costs = np.zeros((ng, ng, ng))

for i, v0 in enumerate(tqdm(v_grid)):
    for j, beta0 in enumerate(beta_grid):
        for k, r0 in enumerate(r_grid):
            u = ctrl.get_ctrl(np.array([v0, beta0, r0, 0, 0]), YREF)
            x_opt, u_opt, cpu_time, cost = ctrl.get_stats()
            # u = np.mean(u_opt[:3,:], axis=0)
            # print(f'u({v0:.2f}, {beta0:.2f}, {r0:.2f}) = {u}')
            deltas[i,j,k] = u[0]
            fxs[i,j,k] = u[1]
            costs[i,j,k] = cost

In [ ]:
# choose a V and plot the slice
min_Fx, max_Fx = np.min(fxs), np.max(fxs)
min_delta, max_delta = np.min(deltas), np.max(deltas)
min_cost, max_cost = np.log(np.min(costs)), np.log(np.max(costs))
for v_idx in range(ng):
    # v_idx = 4
    B, R = np.meshgrid(beta_grid * 180 / π, r_grid * 180 / π)
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1)
    # plt.contourf(beta_grid*180/π, r_grid*180/π, deltas[v_idx,:,:].T, vmin=min_delta, vmax=max_delta)
    plt.scatter(B, R, c=deltas[v_idx,:,:].T, vmin=min_delta, vmax=max_delta)
    plt.colorbar(label='Steering Angle δ (rad)')
    plt.xlabel('Sideslip Angle β (deg)'), plt.ylabel('Yaw Rate r (deg/s)')
    plt.title(f'Steering Angle at v={v_grid[v_idx]:.2f} m/s')
    plt.subplot(1,3,2)
    # plt.contourf(beta_grid*180/π, r_grid*180/π, fxs[v_idx,:,:].T, vmin=min_Fx, vmax=max_Fx)
    plt.scatter(B, R, c=fxs[v_idx,:,:].T, vmin=min_Fx, vmax=max_Fx)
    plt.colorbar(label='Longitudinal Force Fx (N)')
    plt.xlabel('Sideslip Angle β (deg)'), plt.ylabel('Yaw Rate r (deg/s)')
    plt.title(f'Longitudinal Force at v={v_grid[v_idx]:.2f} m/s')
    plt.subplot(1,3,3) # cost plot
    # plt.contourf(beta_grid*180/π, r_grid*180/π, costs[v_idx,:,:].T, vmin=min_cost, vmax=max_cost)
    plt.scatter(B, R, c=np.log(costs[v_idx,:,:].T), vmin=min_cost, vmax=max_cost)
    plt.colorbar(label='MPC Cost')
    plt.xlabel('Sideslip Angle β (deg)'), plt.ylabel('Yaw Rate r (deg/s)')
    plt.title(f'MPC Cost at v={v_grid[v_idx]:.2f} m/s')
    plt.tight_layout()
    plt.show()